In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import vaemof
from vaemof import experiments
from vaemof import utils
from vaemof.vocabs import SELFIESVocab, MOFVocab, PropVocab
from vaemof import modules
from vaemof import training
from vaemof.model import VAEMOF
from vaemof import configs

from vaemof.utils import header_str
vaemof.experiments.plot_settings()

In [7]:
import os
from tqdm.auto import tqdm
import numpy as np
import torch
import rdkit
import pandas as pd
import matplotlib.pyplot as plt

from itertools import product
from collections import OrderedDict, Counter

print(f'rdkit : {rdkit.__version__}')
print(f'torch : {torch.__version__}')
tqdm.pandas()
utils.disable_rdkit_log()

rdkit : 2019.09.3
torch : 1.4.0


## Hparams

In [8]:
preset = 'full'
WORK_DIR = 'results/test'
hparams = configs.get_model_config(WORK_DIR, preset)
print(utils.header_str(preset))
hparams['train_device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
                
testing = configs.testing_config(hparams)
configs.print_config(hparams)
utils.set_seed(hparams['train_seed'])
device = torch.device(hparams['train_device'])

===================================== full =====================================
==================================== config ====================================
== train == :
        train_device:                 cpu
          train_seed:                  42
    train_batch_size:                  64
        train_epochs:                  20
            train_lr:               0.004
     train_clip_grad:                  50
== vae == :
      vae_latent_dim:                 287
           vae_y_dec:                True
     vae_selfies_dec:                True
         vae_mof_enc:                True
         vae_mof_dec:                True
vae_duplicate_smiles:                True
== mof == :
        mof_encoding:                 all
   mof_weighted_loss:                True
         mof_w_start:                 0.0
           mof_w_end:                 1.0
           mof_start:                   0
    mof_const_length:                  10
== y == :
            y_labels:['lcd', 'pld'

## Load MOF data

Assemble smiles formula: [organic core][metal_node][topology][branch_smiles]

In [9]:
smiles_column='branch_smiles'
testtrain_column='train/test'
df = experiments.get_generator_df(csv_file=hparams['files_data'], 
                              smiles_column=smiles_column, 
                              use_duplicates=hparams['vae_duplicate_smiles'],
                              testing=testing)
ids2mofs,mof2ids, mof_columns = experiments.get_mofdict(df,hparams['mof_encoding'])
df.head()

=================================== Testing ====================================
df shape: (713, 9)
df columns: ['index', 'branch_smiles', 'id2mof', 'metal_node', 'mof_index', 'organic_core', 'randomized', 'topology', 'train/test']
Found 713 unique mofs


,index,branch_smiles,id2mof,metal_node,mof_index,organic_core,randomized,topology,train/test
0,0,CC(c1cc(O)cc(O)c1)(C(Sc1cc(O)cc(O)c1)c1cc(O)cc...,0,sym_7_mc_4,4.0,None,False,acs,1
1,2140,O=C1OC2C3(CC([Lr])C2(S(=O)(=O)O)S(=O)(=O)O)C(C...,1,sym_8_mc_7,30198.0,None,False,bcu,1
2,4280,CP(C)c1cc2c(c(P(C)C)c1)N=C(Nc1cc(P(C)C)c([Lr])...,2,sym_8_mc_8,60377.0,None,False,bcu,1
3,6420,CC(c1cc(O)cc(O)c1)(C(Sc1cc(O)cc(O)c1)c1cc(O)cc...,3,sym_8_mc_9,90517.0,None,False,bcu,1
4,8560,O=C1c2cc(-c3cc(O)cc(O)c3)c([Lr])cc2C(c2cc(O)cc...,4,sym_16_mc_6,120699.0,None,False,bcs,1


## Load property data

In [14]:
prop_df = experiments.get_prop_df(csv_file=hparams['files_prop'], 
                              targets = hparams['y_labels'],
                              mof2ids=mof2ids, 
                              testing=testing,
                              smiles_column=smiles_column,
                              compute_scscore=True)
prop_df.head()

=================================== Testing ====================================
Restored variables from data/scscore_1024uint8_model.ckpt-10654.as_numpy.json.gz



Removed 59 datapoints due to mask.
Removed 51 datapoints due non-valid mof (mof2ids).
Removed 4 datapoints due to high selectivity.


,organic_core,metal_node,topology,branch_smiles,lcd,pld,density,avf,avsa,agsa,...,co2ch4_selectivity,co2ch4_heat_avg,co2ch4_heat_co2,co2ch4_heat_ch4,co2ch4_heat_molfrac,selfies_safe,mask,train/test,scscore,id2mof
0,sym_5_vae_317,sym_5_mc_2,ssb,[Lr]CN1C=CC(C=C1)[Lr],14.85666,12.35820,0.547993,0.36595,1879.810,3430.36,...,2.459401,-14.06105,-19.23944,-12.45450,-13.91325,True,True,1,1.810341,430
1,sym_4_on_6,sym_9_mc_5,iac,[Lr]c1ccc(cc1)c1nnc(nn1)c1ccc(cc1)[Lr],25.40998,20.59218,0.228249,0.74805,1068.150,4679.77,...,1.786462,-8.28537,-11.41256,-7.66297,-8.28442,True,True,1,2.841385,182
2,sym_3_on_1,sym_4_mc_1,bor,[Lr]c1ccc(C#Cc2ccc(C#Cc3ccc([Lr])cc3)cc2)cc1,59.72890,44.66190,0.033858,0.94221,301.287,8898.49,...,1.226919,-4.45465,-6.29257,-4.20648,-4.45664,True,True,1,3.273988,15
3,sym_4_on_8,sym_3_mc_0,bor,[Lr]c1cc(C(c2cc([Lr])ccc2C3=O)=O)c3cc1,40.38086,27.66716,0.103757,0.86097,665.363,6412.71,...,1.572181,-6.54100,-9.46598,-6.06977,-6.57509,True,True,0,2.903207,9
4,sym_5_vae_BR911,sym_7_mc_4,stp,[Lr]c1[nH]c2c(n1)cc1c(c2)[nH]c(n1)[Lr],22.97753,18.75747,0.405356,0.53756,1446.400,3568.22,...,2.118090,-8.86976,-13.58480,-7.83311,-8.93193,True,True,1,3.500213,581


## Train/test splits and hparams

In [11]:
train_index  = np.array(df[df[testtrain_column]==1].index.tolist())
test_index  = np.array(df[df[testtrain_column]==0].index.tolist())
prop_train_index  = np.array(prop_df[prop_df[testtrain_column]==1].index.tolist())
prop_test_index  = np.array(prop_df[prop_df[testtrain_column]==0].index.tolist())
print(f'Train sizes: {len(train_index):7d} and {len(prop_train_index):7d}')
print(f'Test  sizes: {len(test_index):7d} and {len(prop_test_index):7d}')

Train sizes:     657 and     802
Test  sizes:      56 and      86


# Vocabulary and preprocessors

In [12]:
smiles_list = df[smiles_column].tolist()+prop_df[smiles_column].tolist()
vocab = SELFIESVocab.from_data(smiles_list)
vocab_mof = MOFVocab.from_data(df.append(prop_df,sort=False),mof_columns,weighting=hparams['mof_weighted_loss'])
vocab_y = PropVocab.from_data(prop_df,hparams['y_labels'], hparams['y_weights'])
vocab, vocab_mof, vocab_y


Alphabet size is 44
Max seq length is 106 with 5 extra padding
Used columns =['metal_node', 'organic_core', 'topology', 'id2mof'] with frequency weighting=True
metal_node   has 15 classes
organic_core has 52 classes
topology     has 41 classes
id2mof       has 713 classes


(<vaemof.vocabs.SELFIESVocab at 0x7f89affcae50>,
 <vaemof.vocabs.PropVocab at 0x7f89affd4790>)

# Instanciate Model

## Careful! (it saves and will overwrite any model previously saved)

In [17]:
utils.clear_torch(model=None)
model = VAEMOF(hparams, vocab, vocab_mof, vocab_y).to(device)
model.save()
modules.model_summary(model, include_children=False)

,Name,Module,Extra,submodule,trainable,n_params,trainable_params
0,z_mu,Linear,"in_features=574, out_features=287, bias=True",False,True,165025,165025
1,z_logvar,Linear,"in_features=574, out_features=287, bias=True",False,True,165025,165025
2,enc_x,CharEncoder,,False,True,380535,380535
3,dec_x,CharDecoder,,False,True,4619620,4619620
4,enc_mof,MOFEncoder,,False,True,483595,483595
5,dec_mof,MOFDecoder,,False,True,319104,319104
6,dec_y,PropDecoder,,False,True,6048,6048


Trainable params: 6137103 out of 6137103 total (100.0%)


## Prepare train/test 

In [18]:
train_mof = model.df_to_tuples(df.loc[train_index],smiles_column)
test_mof =  model.df_to_tuples(df.loc[test_index],smiles_column)
prop_train = model.df_to_tuples(prop_df.loc[prop_train_index],smiles_column)
prop_test = model.df_to_tuples(prop_df.loc[prop_test_index],smiles_column)
train_data = train_mof + prop_train
test_data = test_mof + prop_test

## Train

In [20]:
trainer = training.Trainer(hparams)
trainer.train(model, train_data, test_data)

KeyboardInterrupt: 

# Results
## Load saved model

In [19]:
hparams_file = os.path.join(WORK_DIR,'config.json')
hparams = configs.AttributeDict.from_jsonfile(hparams_file)
model = VAEMOF.load(hparams)

## Training stats

In [22]:
log_df = pd.read_csv(configs.at_results_dir(hparams,'files_log'))
print(log_df.shape)
print(log_df.columns)
log_df.head()

(1, 40)
Index(['epoch', 'test_kl', 'test_x', 'test_mof', 'test_y', 'test_loss', 'lcd',
       'pld', 'density', 'avf', 'avsa', 'agsa', 'co2n2_co2_mol_kg',
       'co2n2_n2_mol_kg', 'co2n2_selectivity', 'co2n2_heat_avg',
       'co2n2_heat_co2', 'co2n2_heat_n2', 'co2n2_heat_molfrac',
       'co2ch4_co2_mol_kg', 'co2ch4_ch4_mol_kg', 'co2ch4_selectivity',
       'co2ch4_heat_avg', 'co2ch4_heat_co2', 'co2ch4_heat_ch4',
       'co2ch4_heat_molfrac', 'scscore', 'lr', 'λ_x', 'λ_kl', 'λ_y', 'λ_mof',
       'train_kl', 'train_x', 'train_mof', 'train_y', 'train_loss',
       'valid_smiles', 'mof_acc', 'mean_r2'],
      dtype='object')


,epoch,test_kl,test_x,test_mof,test_y,test_loss,lcd,pld,density,avf,...,λ_y,λ_mof,train_kl,train_x,train_mof,train_y,train_loss,valid_smiles,mof_acc,mean_r2
0,0,63.596629,2.476707,17.034826,21.914249,2.476707,-0.401462,-0.569074,-0.538268,-0.649688,...,0.0,0.0,105.156191,2.90243,17.126038,34.18994,2.90243,0.0,3.34507,-1.033281


In [ ]:
plt.plot()

In [ ]:
log_file = os.path.join(config['results_dir'],config['log_file'])
log_df = pd.read_csv(log_file)
log_df['recon_weight']=1.0
prefixes = ['kl_','recon_','mof_','y_']
print(log_df.columns)
display(log_df.head(2))
eval_df = log_df.query('mode=="Eval"').set_index('epoch')
train_df = log_df.query('mode=="Train"').set_index('epoch')
plt.plot(train_df['lr'])
plt.title('Learning Rate')
plt.show()
plt.plot(eval_df['valid'])
plt.title('Valid smiles on validation set')
plt.show()

plt.fill_between(eval_df.index, eval_df['y_r2_min'], eval_df['y_r2_max'],alpha=0.25)
plt.plot(eval_df['y_r2_med'],c='g',label='valid')
plt.ylabel('r^2')
plt.show()

for label in prefixes:
    plt.plot(train_df[label+'weight'])
    plt.title(label+' annealer')
    plt.show()

for label in prefixes+['']:
    plt.plot(train_df[label+'loss'],label='Train')
    plt.plot(eval_df[label+'loss'],label='Valid')
    plt.title(label+'Loss')
    plt.yscale("log")
    plt.legend()
    plt.show()

for label in prefixes:
    plt.plot((train_df[label+'weight']*train_df[label+'loss'])/train_df['loss'],label=label)
plt.title('Loss Ratios (Train)')
plt.legend()
plt.show()
for label in prefixes:
    plt.plot((eval_df[label+'weight']*eval_df[label+'loss'])/train_df['loss'],label=label)
plt.title('Loss Ratios (Eval)')
plt.legend()
plt.show()

## Load saved model

In [ ]:
with open(os.path.join(train_dict['results_dir'],config.config_save),'r') as afile:
    config = json.load(afile, object_pairs_hook=models.utils.AttributeDict)

vocab = torch.load(os.path.join(train_dict['results_dir'],config.vocab_save))
vocab_mof = torch.load(os.path.join(train_dict['results_dir'],'mof'))
y_scaler = torch.load(os.path.join(train_dict['results_dir'],'y_scaler'))
model_state = torch.load(os.path.join(train_dict['results_dir'],config.model_save))

model = VAEMOF(vocab, mof_vocab,y_scaler,config).to(device)
model.load_state_dict(model_state)
model = model.to(device)
model.eval()

In [ ]:
print(header_str('Losses'))
log_file = os.path.join(config['results_dir'],config['log_file'])
log_df = pd.read_csv(log_file)
eval_row = log_df.query('mode=="Eval"').iloc[-1]
train_row = log_df.query('mode=="Train"').iloc[-1]
for label in ['kl_','recon_','mof_','y_']+['']:
    min_train = np.min(train_row[label+'loss'])
    min_valid = np.max(eval_row[label+'loss'])
    print('{:6s}loss = {:.3e} / {:.3e}'.format(label,min_train,min_valid))

In [ ]:
sub_sample = 10000
src_data = prop_test
#src_data = random.sample(src_data,min(len(src_data),sub_sample))

predict_properties

n = len(src_data)
batch_size=64
n_loops = int(np.ceil(n/batch_size))
z=[]
y_true=[]
y_pred=[]
with torch.no_grad():
    for chunk in  tqdm(models.utils.chunks(src_data,batch_size),total=n_loops, desc='Generating predictions'):
        x_tensor,mof_tensor,y_tensor,y_mask = trainer.get_collate_fn(model)(chunk)
        _, z_tensor = model.forward_encoder(x_tensor,mof_tensor)
        y_pred_tensor = model.z_to_y(z_tensor)
        z.extend(z_tensor.cpu().numpy())
        y_pred.extend(y_pred_tensor.cpu().numpy())
        y_true.extend(y_tensor.cpu().numpy())
        
z = np.stack(z)
y_true = np.stack(y_true)
y_pred = np.stack(y_pred)
z_pca = PCA(2).fit_transform(z)
z_df = pd.DataFrame()
for index,col in enumerate(targets):
    z_df[col]=y_true[:,index]
    z_df[col+'_pred']=y_pred[:,index]
    r2 = sklearn.metrics.r2_score(y_true[:,index],y_pred[:,index])
    mae = sklearn.metrics.mean_absolute_error(y_true[:,index],y_pred[:,index])
    print('{:20} -> r^2= {:3f}, MAE= {:.3f}'.format(col,r2,mae))
z_df['x']=z_pca[:,0]
z_df['y']=z_pca[:,1]
print(z.shape)

## Prior check

In [ ]:
print(header_str('prior'))
samples = []
n = 1024
batch_size=64
n_loops = int(np.ceil(n/batch_size))
for chunk in  tqdm(models.utils.chunks(list(range(n)),batch_size),total=n_loops, desc='Generating samples'):
    n_batch = min(len(chunk), batch_size)
    z = model.sample_z_prior(n_batch)
    smiles_list = model.sample(n_batch, config.max_length,z=z)
    mof_list = model.z_to_mof(z)
    samples.extend([ [smi]+mof for smi,mof in zip(smiles_list,mof_list)])
    
gen_df = pd.DataFrame(samples,columns=[smiles_column]+mof_columns)
gen_df['valid'] = gen_df[smiles_column].apply(models.utils.valid_smiles)
print('valid smiles: {} out of {} ({}%)'.format(gen_df['valid'].sum(),n,gen_df['valid'].sum()/n*100.0))

gen_df

## Posterior check

In [ ]:
print(header_str('Posterior check'))

tries=2
sub_sample =1000
src_data = train_data
src_data = random.sample(src_data,min(len(src_data),sub_sample))
valid_smiles=[]
recon_smiles=[]
n = len(src_data)
results=[]
mof_results=[]
print(n)
with torch.no_grad():
    for t in tqdm(src_data):
        batch = [t]*tries
        true_smiles = isosmiles(vocab.ids2string(t[0]))
        true_mof = mof_vocab.ids2mof(t[1])
        x_tensor,mof,y,y_mask = trainer.get_collate_fn(model)(batch)
        _, z_tensor = model.forward_encoder(x_tensor,mof)
        mof_list = np.array(model.z_to_mof(z_tensor))
        pred_mof = [mof_list[:,i].tolist() for i in range(3)]
        acc_mof = any([ all(i==true_mof) for i in mof_list])
        _, x_recon_tensor = model.forward_x_decoder(x_tensor,z_tensor)
        re_smiles = [model.tensor2string(x_i) for x_i in x_recon_tensor]
        valid_smiles = [si for si in set(re_smiles) if models.utils.valid_smiles(si)]
        valid_smiles = [isosmiles(si) for si in valid_smiles]
        same_smiles = [si for si in valid_smiles if si==true_smiles]
        results.append([true_smiles, re_smiles[0], len(same_smiles)>0,len(valid_smiles)>0])
        mof_results.append(acc_mof)

        
post_df = pd.DataFrame(results,columns=['smiles','recon_smiles','same','valid'])
print('valid: {} out of {} ({:.2f}%)'.format(post_df['valid'].sum(),n,post_df['valid'].sum()/n*100.0))
print('same : {} out of {} ({:.2f}%)'.format(post_df['same'].sum(),n,post_df['same'].sum()/n*100.0))
display(post_df.head(1))
print('Mof recon acc: {}'.format(float(sum(mof_results))/float(len(mof_results))))

In [ ]:
sub_sample = 10000
src_data = train_data
src_data = random.sample(src_data,min(len(src_data),sub_sample))
n = len(src_data)
batch_size=64
n_loops = int(np.ceil(n/batch_size))
z=[]
mof=[]
mof_ids=[]
with torch.no_grad():
    for chunk in  tqdm(models.utils.chunks(src_data,batch_size),total=n_loops, desc='Generating samples'):
        mof.extend([mof_vocab.ids2mof(t[1]) for t in chunk])
        x_tensor,mof_tensor,y, y_mask = trainer.get_collate_fn(model)(chunk)
        _, z_tensor = model.forward_encoder(x_tensor,mof_tensor)
        z.extend(z_tensor.cpu().numpy())
        mof_ids.extend([mof_vocab.ids2mof(t) for t in mof_tensor.cpu().numpy()])
        
z = np.stack(z)
z_pca = PCA(2).fit_transform(z)
z_df = pd.DataFrame(mof,columns=mof_columns)
z_df['x']=z_pca[:,0]
z_df['y']=z_pca[:,1]
mof_ids = np.stack(mof_ids)
print(z.shape,mof_ids.shape)

In [ ]:
top_k=8
for index,col in enumerate(mof_columns):
    print(models.utils.header_str(col))
    plt.figure(figsize=(8,8))
    top_cat = z_df[col].value_counts(sort=True).iloc[:top_k].index.tolist()
    sns.scatterplot(x='x',y='y',hue=col,s=20,data=z_df[z_df[col].isin(top_cat)])
    plt.legend(bbox_to_anchor=(1.0, .5))
    plt.title(col)
    plt.show()